In [1]:
import os

In [2]:
%pwd

'c:\\Users\\sandr\\PROJECTS_CODE\\car_sales_data_science_workflow\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\sandr\\PROJECTS_CODE\\car_sales_data_science_workflow'

In [33]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    transformed_data: Path

In [34]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [35]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            transformed_data=config.transformed_data
        )

        return data_transformation_config

In [46]:
import os
from mlProject import logger
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

In [51]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        
        self.config = config
        self.data = pd.read_csv(self.config.data_path)

    
    def cleaning_data(self):

        df = self.data.drop(['Order','Sales_in_thousands','Order Date','dealership','color','Vehicle_type'], axis=1)
        #remove letters from strings in "Price_in_lakhs" column
        df['Price_in_lakhs'] = df['Price_in_lakhs'].str.replace('L', '', regex=True)   
        logger.info("Removed unrelated columns")
        logger.info(df.shape)
        print(df.shape)

        df.to_csv(os.path.join(self.config.root_dir, "transformed_data.csv"),index = False)
        logger.info("Transformed data created")

    def train_test_spliting(self):
        data = pd.read_csv(self.config.transformed_data)

        # Split the data into training and test sets. (0.75, 0.25) split.
        train, test = train_test_split(data)

        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)

        logger.info("Splited data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)

        print(train.shape)
        print(test.shape)

In [52]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.cleaning_data()
    data_transformation.train_test_spliting()
    # final_df.head()
except Exception as e:
    raise e

[2023-08-02 20:28:59,566: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-02 20:28:59,567: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-02 20:28:59,569: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-02 20:28:59,570: INFO: common: created directory at: artifacts]
[2023-08-02 20:28:59,571: INFO: common: created directory at: artifacts/data_transformation]
[2023-08-02 20:28:59,576: INFO: 816391733: Removed unrelated columns]
[2023-08-02 20:28:59,577: INFO: 816391733: (1000, 11)]
(1000, 11)
[2023-08-02 20:28:59,583: INFO: 816391733: Transformed data created]
[2023-08-02 20:28:59,602: INFO: 816391733: Splited data into training and test sets]
[2023-08-02 20:28:59,603: INFO: 816391733: (750, 11)]
[2023-08-02 20:28:59,604: INFO: 816391733: (250, 11)]
(750, 11)
(250, 11)
